In [1]:
from utils import loadGloveModel, save_images
from models import build_generator_func
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
import tensorflow as tf
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy.stats import entropy
from scipy.linalg import sqrtm

/Users/rajibdas/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
GENERATE_RES = 2 
GENERATE_SQUARE = 32 * GENERATE_RES 
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 1
PREVIEW_COLS = 1
PREVIEW_MARGIN = 1

In [95]:
glove_embeddings = loadGloveModel("/Users/rajibdas/Workspace/UB/CV/data/glove.6B.300d.txt")

Loading Glove Model
Done. 400000  words loaded!


In [ ]:

def test_image(text,num):
    test_embeddings = np.zeros((1,300),dtype=np.float32)

    x = text.lower()
    x = x.replace(" ","")
    count = 0
    for t in x:
        try:
            test_embeddings[0] += glove_embeddings[t]
            count += 1
        except:
            pass
    test_embeddings[0] /= count
    test_embeddings =  np.repeat(test_embeddings,[28],axis=0)
    noise = tf.random.normal([28, 100])
    save_images(generator, num,noise,test_embeddings, PREVIEW_MARGIN, PREVIEW_ROWS, PREVIEW_COLS,GENERATE_SQUARE)

def load_and_preprocess_image(image_path):
    # Load the image file, resizing it to 299x299 pixels
    img = load_img(image_path, target_size=(299, 299))
    img = img_to_array(img)
    img = preprocess_input(img)  # Preprocess the image using InceptionV3's required preprocessing
    return img

def inception_score(images, model, batch_size=32):
    preds = []
    for i in range(0, len(images), batch_size):
        batch = images[i:i + batch_size]
        preds.append(model.predict(batch))
    preds = np.concatenate(preds, axis=0)
    # Calculate the marginal probability
    marginal_prob = np.mean(preds, axis=0)
    # Calculate the KL divergence and then the score
    scores = entropy(preds, marginal_prob, base=2, axis=1)
    return np.exp(np.mean(scores))

def calculate_fid(model, real_images, generated_images):
    act1 = model.predict(real_images)
    act2 = model.predict(generated_images)
    # Calculate mean and covariance
    mu1, sigma1 = np.mean(act1, axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = np.mean(act2, axis=0), np.cov(act2, rowvar=False)
    # Calculate the difference and trace for FID
    ssdiff = np.sum((mu1 - mu2) ** 2.0)
    covmean = sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid

## Flowers

In [30]:
generator = build_generator_func(100,300, 3)
generator.load_weights("/Users/rajibdas/Workspace/UB/CV/Text-to-Image-Synthesis/GAN2/gan-flower-model/text_to_image_generator_character.h5")

In [68]:
flower_meta = pd.read_csv("../../data/flowers/metadata.csv")

In [69]:
captions = flower_meta.iloc[-100:]['caption']
original_images = flower_meta.iloc[-100:]['file_name']

In [76]:
generated_image_paths = []
real_image_paths = []
for i, caption in enumerate(captions):
    test_image(caption, i)
    generated_image_paths.append("/Users/rajibdas/Workspace/UB/CV/Text-to-Image-Synthesis/GAN2/output/"+f"train-{i}.png")
    real_image_paths.append("/Users/rajibdas/Workspace/UB/CV/data/flowers/"+original_images.iloc[i])

1/1 [==============================] - 1s 504ms/step


In [91]:
# Convert paths to preprocessed image data
real_images = np.array([load_and_preprocess_image(path) for path in real_image_paths])
generated_images = np.array([load_and_preprocess_image(path) for path in generated_image_paths])

In [99]:
# Load InceptionV3 model
model_is = tf.keras.applications.InceptionV3(include_top=True, input_shape=(299, 299, 3))

# Calculate Inception Score
is_score = inception_score(generated_images, model_is)
print("Inception Score:", inception_score)

1/1 [==============================] - 1s 669ms/step
Inception Score: 6.463261


In [109]:
# Initialize InceptionV3 for FID with pooling
model_fid = tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Calculate FID
fid_score = calculate_fid(model_fid, real_images, generated_images)
print("FID:", fid_score)


4/4 [==============================] - 4s 851ms/step
FID: 64.23512


## Buds

In [101]:
generator = build_generator_func(100,300, 3)
generator.load_weights("/Users/rajibdas/Workspace/UB/CV/Text-to-Image-Synthesis/GAN2/gan-cub-model/text_to_image_generator_character.h5")

In [102]:
GENERATE_RES = 2 
GENERATE_SQUARE = 32 * GENERATE_RES 
IMAGE_CHANNELS = 3

# Preview image 
PREVIEW_ROWS = 1
PREVIEW_COLS = 1
PREVIEW_MARGIN = 1

In [103]:
cub_meta = pd.read_csv("../../data/cubs/cubs/metadata.csv")

In [104]:
captions = cub_meta.iloc[-100:]['caption']
original_images = cub_meta.iloc[-100:]['file_name']

In [105]:
generated_image_paths = []
real_image_paths = []
for i, caption in enumerate(captions):
    test_image(caption, i)
    generated_image_paths.append("/Users/rajibdas/Workspace/UB/CV/Text-to-Image-Synthesis/GAN2/output/"+f"train-{i}.png")
    real_image_paths.append("/Users/rajibdas/Workspace/UB/CV/data/cubs/cubs/"+original_images.iloc[i])

1/1 [==============================] - 0s 487ms/step


In [106]:
# Convert paths to preprocessed image data
real_images = np.array([load_and_preprocess_image(path) for path in real_image_paths])
generated_images = np.array([load_and_preprocess_image(path) for path in generated_image_paths])

In [107]:
# Load InceptionV3 model
model_is = tf.keras.applications.InceptionV3(include_top=True, input_shape=(299, 299, 3))

# Calculate Inception Score
is_score = inception_score(generated_images, model_is)
print("Inception Score:", 3.24894)

1/1 [==============================] - 1s 610ms/step
Inception Score: 3.24894


In [108]:
# Initialize InceptionV3 for FID with pooling
model_fid = tf.keras.applications.InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

# Calculate FID
fid_score = calculate_fid(model_fid, real_images, generated_images)
print("FID:", fid_score)


4/4 [==============================] - 5s 1s/step
FID: 71.245232
